In [1]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ha" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/hausa_mt/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/hausa_mt/%s-%s-%s" % (source_language, target_language, tag)

In [3]:

!echo $gdrive_path

/content/drive/My Drive/hausa_mt/en-ha-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.2MB/s 


In [5]:

# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   2 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ha.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  24 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ha.zip

 289 MB Total size
./JW300_latest_xml_en-ha.xml.gz ... 100% of 2 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ha.zip ... 100% of 24 MB


In [0]:
# # Download the global test set.
# #! wget https://github.com/WalePhenomenon/Hausa-NMT/blob/master/test.en
  
# # And the specific test set for this language pair.
# os.environ["trg"] = target_language 
# os.environ["src"] = source_language 

# ! wget https://github.com/WalePhenomenon/Hausa-NMT/blob/master/test.en
# #! mv test.en test.en
# ! wget https://github.com/WalePhenomenon/Hausa-NMT/blob/master/test.ha 
# #! mv test.ha test.ha

In [0]:
# # Read the test data to filter from train and dev splits.
# # Store english portion in set for quick filtering checks.
# en_test_sents = set()
# filter_test_sents = "test.en"
# j = 0
# with open(filter_test_sents) as f:
#   for line in f:
#     en_test_sents.add(line.strip())
#     j += 1
# print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

In [8]:

import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df1= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df1.shape)
df1.head(3)


Loaded data and skipped 0/237064 lines since contained in test set.
(237065, 2)


,source_sentence,target_sentence
0,Questions From Readers,Tambayoyi Daga Masu Karatu
1,How seriously should Christians view an engage...,Yaya ya kamata Kiristoci su ɗauki riƙo ?
2,An engagement to marry is a cause for happines...,"Riƙo dalili ne na farinciki , amma kuma muhimm..."


In [9]:
df1.shape

(237065, 2)

In [0]:

# drop duplicate translations
#df_jw300_pp = df_jw300.drop_duplicates()
df1_pp = df1.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
#df_pp.drop_duplicates(subset='source_sentence', inplace=True)
#df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
#df_jw300_pp = df_jw300_pp.sample(frac=1, random_state=seed).reset_index(drop=True)
df1_pp = df1_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [0]:
# # Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# # test and training sets.
# ! pip install fuzzywuzzy
# ! pip install python-Levenshtein
# import time
# from fuzzywuzzy import process
# import numpy as np

# # reset the index of the training set after previous filtering
# df_pp.reset_index(drop=False, inplace=True)

# # Remove samples from the training data set if they "almost overlap" with the
# # samples in the test set.

# # Filtering function. Adjust pad to narrow down the candidate matches to
# # within a certain length of characters of the given sample.
# def fuzzfilter(sample, candidates, pad):
#   candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
#   if len(candidates) > 0:
#     return process.extractOne(sample, candidates)[1]
#   else:
#     return np.nan

# # NOTE - This might run slow depending on the size of your training set. We are
# # printing some information to help you track how long it would take. 
# scores = []
# start_time = time.time()
# for idx, row in df_pp.iterrows():
#   scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
#   if idx % 1000 == 0:
#     hours, rem = divmod(time.time() - start_time, 3600)
#     minutes, seconds = divmod(rem, 60)
#     print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# # Filter out "almost overlapping samples"
# df_pp['scores'] = scores
# df_pp = df_pp[df_pp['scores'] < 95]

In [0]:
# # df_jw300_pp['en_word_count'] = df_jw300_pp['source_sentence'].str.count(' ') + 1
# # df_jw300_pp['ha_word_count'] = df_jw300_pp['target_sentence'].str.count(' ') + 1

# df_all_pp['en_word_count'] = df_all_pp['source_sentence'].str.count(' ') + 1
# df_all_pp['ha_word_count'] = df_all_pp['target_sentence'].str.count(' ') + 1

In [0]:
# #Compute Sum
# print(df_jw300_pp['en_word_count'].sum())
# print(df_jw300_pp['ha_word_count'].sum())
# #print(df_all_pp['en_word_count'].sum())
# #print(df_all_pp['ha_word_count'].sum())

In [0]:
# #Compute Mean and standard deviation

# print(df_jw300_pp['en_word_count'].mean())
# print(df_jw300_pp['en_word_count'].std())

# print(df_jw300_pp['ha_word_count'].mean())
# print(df_jw300_pp['ha_word_count'].std())

# # print(df_all_pp['en_word_count'].mean())
# # print(df_all_pp['en_word_count'].std())

# # print(df_all_pp['ha_word_count'].mean())
# # print(df_all_pp['ha_word_count'].std())

In [0]:
# #df_jw300_pp = df_jw300_pp[['source_sentence', 'target_sentence']]
# df_all_pp = df_all_pp[['source_sentence', 'target_sentence']]

In [17]:
#Load train, test, dev from local drive

from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving dev.csv to dev.csv


In [0]:
import io
import pandas as pd
dev_df = pd.read_csv(io.BytesIO(uploaded['dev.csv']))
test_df = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:
test_df = test_df[['source_sentence', 'target_sentence']]

dev_df = dev_df[['source_sentence', 'target_sentence']]

In [20]:
stripped_df = pd.concat([df1_pp, test_df, dev_df])
stripped_df = stripped_df.drop_duplicates(keep = False)
stripped_df.shape

(222650, 2)

In [21]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv
import numpy as np

# # Do the split between dev/train and create parallel corpora
# num_dev_patterns = 1000

# # Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
# if lc:  # Julia: making lowercasing optional
#     df_all_pp["source_sentence"] = df_all_pp["source_sentence"].str.lower()
#     df_all_pp["target_sentence"] = df_all_pp["target_sentence"].str.lower()

# #Divide df_pp into train and test sets

# msk = np.random.rand(len(df_all_pp)) < 0.99
# train = df_all_pp[msk]
# test_df = df_all_pp[~msk]

# # Further divide train into train and additional dev set which should be the last 1000
# dev_df = train.tail(num_dev_patterns) # Herman: Error in original
# stripped_df = train.drop(train.tail(num_dev_patterns).index)
with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test_df.iterrows():
    src_file.write(str(row["source_sentence"])+"\n")
    trg_file.write(str(row["target_sentence"])+"\n")

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped_df.iterrows():
    src_file.write(str(row["source_sentence"])+"\n")
    trg_file.write(str(row["target_sentence"])+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev_df.iterrows():
    src_file.write(str(row["source_sentence"])+"\n")
    trg_file.write(str(row["target_sentence"])+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*
! head test.*

==> train.en <==
When you experience any kind of anxiety , the Bible’s wise advice is to “ throw your burden on Jehovah . ”
13 , 14 . ( a ) How could Satan use our choice of health treatments to breach our integrity ?
Anointed ones “ have been given a new birth , not by corruptible , but by incorruptible reproductive seed , through the word of the living and enduring God . ”
Others took vows as Nazirites .
To that end , as we read God’s Word , how can we make sure that our study is productive ?
Is that always wise ?
At one point , Paul and his companions returned to Syrian Antioch .
All Christians ought to be alert to discern what their prayers reveal .
Instead , let us be determined to remain loyal to Jehovah and to the organization that has a long record of quenching our thirst with the pure and refreshing waters of truth from God’s inspired Word . ​ — Isa .
Despite the many troubles that Paul experienced , including “ dangers from robbers , ” he lived by his words , never putting pe

In [0]:
stripped_df.to_csv('train.csv')
!cp train.csv drive/My\ Drive/hausa_mt/

In [0]:
test_df.to_csv('test.csv')
!cp test.csv drive/My\ Drive/hausa_mt/

In [0]:
dev_df.to_csv('dev.csv')
!cp dev.csv drive/My\ Drive/hausa_mt/

In [25]:

# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2380 (delta 0), reused 1 (delta 0), pack-reused 2377
Receiving objects: 100% (2380/2380), 2.60 MiB | 2.30 MiB/s, done.
Resolving deltas: 100% (1670/1670), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 276kB 7.2MB/s 
     |████████████████████████████████| 307kB 40.3MB/s 
     |████████████████████████████████| 17.1MB 213kB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 215kB 47.5MB/s 
     |████████████████████████████████| 747kB 38.9MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73768 sha256=0d6913d6217f1d56f920383b9860865808fd3ac12771a22d8e16d73135a9a4ab
  Stored in directory: /tmp/pip-ephem-wheel-cach

In [26]:
###### IOHAVOC MODIFICATIONS ==>> CREATE THE VOCAB FOR NON-BPE EXPERIMENTS
from os import path

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman!

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.$src joeynmt/data/$src$tgt/train.$tgt --output_path joeynmt/data/$src$tgt/vocab-nonBPE.txt


# Some output
! echo "Hausa test Sentences"
! tail -n 5 test.$tgt
! echo "Combined Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab-nonBPE.txt  

dev.csv  dev.ha    test.en  train.csv  train.ha
dev.en	 test.csv  test.ha  train.en
dev.csv  dev.ha    test.en  train.csv  train.ha
dev.en	 test.csv  test.ha  train.en
Hausa test Sentences
( Zabura 101 : 2 ) Hakika , ana iya gwada amincinmu sa’ad da muke cikin gida .
[ Hoto a shafi na 6 ]
Saboda haka , zai dace mu horar da kanmu sosai yanzu da ba ma fuskantar farmaki .
( 2 Timothawus 3 : 14 , 15 ) Tun sa’ad da Timoti yake yaro , uwarsa da kakarsa suka kafa masa tushe mai ƙarfi a bangaskiyarsa ga Allah bisa ga sani na Nassosi Masu Tsarki .
‘ ’ yancin rubuta da kuma faɗin abin da suke ganin ya dace da kuma idar da saƙonni ga mutane . ’ — Talifi na 10 .
Combined Vocab
Bintu’s
intrusions
Keith’s
citations
retaliated
Impulsively
dissolving
switches
Alicia’s
Uzairu


In [0]:
# # One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# # Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# # Do subword NMT
# from os import path
# os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
# os.environ["tgt"] = target_language

# # Learn BPEs on the training data.
# os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
# ! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# # Apply BPE splits to the development and test data.
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# # Create directory, move everyone we care about to the correct location
# ! mkdir -p $data_path
# ! cp train.* $data_path
# ! cp test.* $data_path
# ! cp dev.* $data_path
# ! cp bpe.codes.4000 $data_path
# ! ls $data_path

# # Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

# # Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# # Some output
# ! echo "BPE Hausa Sentences"
# ! tail -n 5 test.bpe.$tgt
# ! echo "Combined BPE Vocab"
# ! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train"
    dev:   "data/{name}/dev"
    test:  "data/{name}/test"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab-nonBPE.txt"
    trg_vocab: "data/{name}/vocab-nonBPE.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [29]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-04-27 01:03:51,778 Hello! This is Joey-NMT.
2020-04-27 01:03:51.924808: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04-27 01:03:53,782 Total params: 28592128
2020-04-27 01:03:53,784 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [0]:

# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -Lr joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"


In [31]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 108395.74219	PPL: 122.17014	bleu: 0.34286	LR: 0.00030000	*
Steps: 2000	Loss: 94202.69531	PPL: 65.11836	bleu: 2.09676	LR: 0.00030000	*
Steps: 3000	Loss: 84927.75781	PPL: 43.16496	bleu: 3.94829	LR: 0.00030000	*
Steps: 4000	Loss: 77539.92969	PPL: 31.10942	bleu: 5.71916	LR: 0.00030000	*
Steps: 5000	Loss: 72427.96094	PPL: 24.80106	bleu: 7.68173	LR: 0.00030000	*
Steps: 6000	Loss: 67873.53125	PPL: 20.26669	bleu: 9.45125	LR: 0.00030000	*
Steps: 7000	Loss: 64469.89062	PPL: 17.42817	bleu: 12.14703	LR: 0.00030000	*
Steps: 8000	Loss: 61863.07422	PPL: 15.52610	bleu: 12.79696	LR: 0.00030000	*
Steps: 9000	Loss: 59382.30078	PPL: 13.90911	bleu: 13.47694	LR: 0.00030000	*
Steps: 10000	Loss: 57780.90625	PPL: 12.95590	bleu: 14.73205	LR: 0.00030000	*
Steps: 11000	Loss: 56000.54297	PPL: 11.97264	bleu: 13.53949	LR: 0.00030000	*
Steps: 12000	Loss: 54810.96094	PPL: 11.35760	bleu: 16.23534	LR: 0.00030000	*
Steps: 13000	Loss: 53275.18359	PPL: 10.61006	bleu: 15.64293	LR: 0.00030000	*
Steps: 14000

In [32]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-04-27 06:31:14,770 Hello! This is Joey-NMT.
2020-04-27 06:32:51,053  dev bleu:  28.71 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-04-27 06:34:34,330 test bleu:  28.06 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [33]:
python3 plot_validation.py "$gdrive_path/models/${src}${tgt}_transformer/" --plot_values bleu PPL --output_path my_plot.pdf

SyntaxError: ignored